In [ ]:
import holoviews as hv
import hvplot.pandas
import intake
import panel as pn

hv.streams.PlotSize.scale = 0.2

pn.extension('tabulator', design='material', template='material', theme='dark')

In [ ]:
cat = intake.open_anaconda_catalog('examples')['uswtdb']

df = pn.state.as_cached('windturbines', cat.wind_turbines.read)

df['x'], df['y'] = hv.element.tiles.lon_lat_to_easting_northing(df.xlong, df.ylat)

df.head()

In [ ]:
ls = hv.link_selections.instance()

plot = df.hvplot.points('x', 'y', rasterize=True, cmap='viridis', cnorm='eq_hist', xaxis=None, yaxis=None, tiles='CartoDark', responsive=True, min_height=500)

plot_ls = ls(plot)

plot_ls

In [ ]:
df_sel = ls.selection_param(df)

dfi_sel = hvplot.bind(lambda df: df, df_sel).interactive()[['t_state', 't_county', 't_manu', 'p_year', 'p_name', 'p_cap']]

table = pn.widgets.Tabulator(dfi_sel, pagination='remote', page_size=10, sizing_mode='stretch_width', show_index=True)

table

In [ ]:
groupby = pn.widgets.RadioButtonGroup(options=['p_year', 't_state', 't_manu'])
topn = pn.widgets.IntSlider(start=5, end=50, name='Top N')

agg_bar_plot = dfi_sel.groupby(groupby)[['p_cap']].sum().sort_values('p_cap', ascending=False).iloc[:topn].hvplot.bar(rot=45)

agg_bar_plot

In [ ]:
pn.Column(
    plot_ls,
    pn.Row(
        table, agg_bar_plot
    )
).servable(title='Windturbine Dashboard')